In [1]:
# Libraries import 
import pdbufr
import sys
import traceback
 
from math import isnan
from eccodes import *
from ecmwf.opendata import Client
from datetime import datetime, timedelta
from PIL import Image

import os
import birdy
import pandas as pd
import numpy as np
import xarray as xr
import requests

from ipywidgets import interact
import ipyleaflet
import ipywidgets as widgets

from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

from tracks_utils import *
from atm_utils import * 

In [2]:
## WIDGET TO IMPORT THE TITLE IMAGE ##

file = open("layout_data/title.jpg", "rb")
image = file.read()
title_img = widgets.Image(
    value=image,
    format='jpg',
)

In [3]:
## WIDGET TO DEFINE THE STARTING DATE OF THE FORECAST ##
## This sets the date of which we want to download the forecast ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

In [4]:
## DOWNLOAD THE FORECAST TRACKS ##
# The forecast data is saved as bufr file called tc_test_track_data.bufr    
download_tracks_forecast(start_date_forecast.value)

## DOWNLOAD THE OBSERVED TRACKS DATA FROM IBTrACS ##
# The observed data is saved as a csv file called ibtracs.ACTIVE.list.v04r00.csv
url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

20230904000000-240h-enfo-tf.bufr:   0%|          | 0.00/618k [00:00<?, ?B/s]

In [5]:
## LOAD THE FORECAST DATA OF THE TRACKS IN A DATAFRAME ##
# create_storms_df loads a file called tc_test_track_data.bufr
df_storms_forecast = create_storms_df()

## LOAD THE OBSERVED TRACKS IN A DATAFRAME ## 
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

## CREATE THE LIST OF ACTIVE CYCLONES IN THE FORECAST ##
cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]

In [6]:
## WIDGET TO SHOW THE LIST OF ACTIVE CYCLONES AND SELECT THE ONE OF WHICH WE WANT TO PLOT THE DATA ##

cyclone = widgets.Dropdown(
    options = active_cyclones,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Update storms list when downloading new tracks data
def update_storms_list(_):
    download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = create_storms_df()
    df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
    cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
    active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]
    cyclone.options = active_cyclones

start_date_forecast.observe(update_storms_list, names='value')

In [7]:
## WIDGET TO SELECT WHICH ENSEMBLE MEMBERS OF THE FORECAST TRACK PLOT ##

members = df_storms_forecast.ensembleMemberNumber.unique().tolist()
ens_members = widgets.SelectMultiple(
    options=members,
    value=[1],
    description="Ensemble members to plot:",
    disable=False
)

ens_members.style.description_width = '168px'

# Print widget
message = widgets.HTML(
    value="Multiple values can be selected with <b>shift</b> and/or <b>ctrl</b> (or <b>command</b>)",
)

In [8]:
## UPDATE BUTTON TO PLOT NEW TRACKS AND DATA DEPENDING ON THE CYCLONE SELECTED ##

update_button = widgets.Button(
    description = 'Update Forecast',
    icon = "check",
    style=dict(
        button_color='blue',
        font_weight='bold',
        text_color='lightgreen',
        text_decoration='underline',
))

update_output = widgets.Output()

# Plot new data when clicking the Update Forecast Button
def on_button_clicked(b):
    with update_output:
        ## Output of initial lat-lon, average track and initial-final time step of the cyclone for the next sections ##
        code, name = cyclone.value.split('-')
        display(widgets.HTML(value=f"Creating the dataset for tropical cyclone <b>{name}</b>"), clear=True) # clear previous cyclones plots
        df_storms_forecast = create_storms_df()
        df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
        df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
        df_f.reset_index(drop=True, inplace=True)
        df_o = df_storms_observed[df_storms_observed.NAME.squeeze() == name]
        df_o.reset_index(drop=True, inplace=True)
        # Initial and final lat-lon
        initial_lat_lon = (df_f.latitude.iloc[0], df_f.longitude.iloc[0])
        final_lat_lon = (df_f.latitude.iloc[-1], df_f.longitude.iloc[-1])
        coord = ((initial_lat_lon[0] - final_lat_lon[0])/2, (initial_lat_lon[1] - final_lat_lon[1])/2)
        # Average track
        locations_avg, timesteps_avg = mean_forecast_track(df_f)
        # Initial and final time step
        df_f["date"] = datetime(df_f.year[0], df_f.month[0], df_f.day[0], df_f.hour[0]) + timedelta(hours=1) * df_f.timePeriod
        initial_timestep = datetime.strptime(df_o.ISO_TIME.squeeze()[0], "%Y-%m-%d %H:%M:%S")
        final_timestep = df_f.date.iloc[-1].to_pydatetime()
        display(widgets.HTML(value="Dataset prepared!"))
        
        ## Plots of the tracks ##
        display(widgets.HTML(value=f"Creating the plots for the tropical cyclone tracks considering ensemble members: <b>{ens_members.value}</b>"))
        tc_track_map = plot_cyclone_tracks_ipyleaflet(ens_members.value, df_f, df_o)
        display(tc_track_map)
        
        # print("Forecast updated!", end='\r')

update_button.on_click(on_button_clicked)

In [9]:
## GROUP THE THREE PRINCIPAL WIDGET IN A VERTICAL BOX STRUCTURE ##
selection_box = widgets.Box([start_date_forecast, cyclone, message, ens_members],
                  layout=widgets.Layout(
                      display='flex',
                      flex_flow='column',
                      border='3px dashed orange',
                      align_items='center',
                      width='100%'
                  ))

## GROUP THE THREE PRINCIPAL WIDGET AND THE UPDATE BUTTON TOGETHER
widgets_box = widgets.Box([selection_box, update_button],
                          layout=widgets.Layout(
                              display='flex',
                              flex_flow='column',
                              align_items='center',
                              # align_content='space-between',
                              # width='100%',
                              # height='100%'
                          ))


## GROUP THE TITLE IMAGE WITH THE GENERAL WIDGETS BOX ##
title_box = widgets.Box([title_img, widgets_box],
                        layout=widgets.Layout(
                            display='flex',
                            flex_flow='row',
                            align_items='center'
                        ))

display(title_box, update_output)

Box(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\xcd\x00\xcd\x00\x00\xff\xe2\x0f\x…

Output()

In [10]:
#grid = widgets.GridspecLayout(2,2)
#grid[0,:] = title_box
#grid[1,:] = update_output

#grid[1,:].laout.height = 'auto'
#grid